In [2]:
from google.colab import drive
drive.mount('/content/drive')

^C
Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.

  Using cached soundfile-0.12.1-py2.py3-none-win_amd64.whl (1.0 MB)
  Using cached cffi-1.15.1-cp310-cp310-win_amd64.whl (179 kB)
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)
^C
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\fujitsu\\Desktop\\Georgia\\CS7643_DeepLearning\\whisper\\.conda\\Lib\\site-packages\\~umpy\\.libs\\libopenblas64__v0.3.23-gcc_10_3_0.dll'
Consider using the `--user` option or check the permissions.



  Using cached librosa-0.10.0.post2-py3-none-any.whl (253 kB)
  Using cached audioread-3.0.0-py3-none-any.whl
  Using cached scipy-1.11.1-cp310-cp310-win_amd64.whl (44.0 MB)
  Using cached scikit_learn-1.3.0-cp310-cp310-win_amd64.whl (9.2 MB)
  Using cached joblib-1.3.1-py3-none-any.whl (301 kB)
  Using cached numba-0.57.1-cp310-cp310-win_amd64.whl (2.5 MB)
  Using cached pooch-1.6.0-py3-none-any.whl (56 kB)
  Using cached soxr-0.3.5-cp310-cp310-win_amd64.whl (184 kB)
  Using cached lazy_loader-0.3-py3-none-any.whl (9.1 kB)
  Using cached msgpack-1.0.5-cp310-cp310-win_amd64.whl (61 kB)
  Using cached llvmlite-0.40.1-cp310-cp310-win_amd64.whl (27.7 MB)
  Using cached numpy-1.24.4-cp310-cp310-win_amd64.whl (14.8 MB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.1
    Uninstalling numpy-1.25.1:
      Successfully uninstalled numpy-1.25.1

  Usi

In [ ]:
%cd '/content/drive/MyDrive/adapter-transformers'
!ls

In [3]:
%pip install -U transformers
%pip install -U accelerate
%pip install importlib-metadata
%pip install soundfile
%pip install librosa
%pip install jiwer
%pip install evaluate
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import evaluate
import accelerate
from transformers import WhisperTokenizer, WhisperFeatureExtractor, WhisperProcessor
from src.transformers.models.whisper.modeling_whisper import WhisperForConditionalGeneration
from src.transformers.adapters import AdapterConfig
from transformers import Trainer, TrainingArguments, Seq2SeqTrainingArguments
from datasets import load_dataset, DatasetDict, Audio
from dataclasses import dataclass
from typing import Any, Dict, List, Union

ModuleNotFoundError: No module named 'transformers.utils.versions'

In [ ]:
model_name = "openai/whisper-small"

In [ ]:
#Load Dataset
common_voice = DatasetDict()
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="train+validation")
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="test")
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print('###############Loading Dataset Is Complete################')


In [ ]:
#Extract, Tokenize and Process
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
tokenizer = WhisperTokenizer.from_pretrained(model_name, language="Hindi", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")

print('###############Extract, Tokenize and Process Is Complete################')

In [ ]:
#Prepare Data
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1)

print('###############Data Preparation Is Complete################')

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

print('###############Data Collator Is Complete################')

In [ ]:
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

print('###############Compute metrics Is Complete################')

In [ ]:
# Define the adapter layers
model = WhisperForConditionalGeneration.from_pretrained(model_name)
adapter_config = AdapterConfig.load("pfeiffer")
model.add_adapter("transcribe")

print('###############Adapter Inclusion Is Complete################')


In [ ]:
# Set up your training arguments and data
training_args = TrainingArguments(
    #max_steps=80000,
    output_dir="./results",
    num_train_epochs=6,
    learning_rate=3e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    logging_steps=1000,
    overwrite_output_dir=True,
    remove_unused_columns=False
    #save_total_limit=2,
)

# Create a Trainer and train the model on your task-specific dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

print('###############Training Arguments placing Is Complete################')

In [ ]:
trainer.train()
print('###############Training Is Complete################')

In [ ]:
trainer.evaluate()

In [ ]:
%python adapter.py